In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [19]:
pickle_file = 'data_for_recognition/pickled_data'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (210, 15, 15) (210,)
Validation set (45, 15, 15) (45,)
Test set (45, 15, 15) (45,)


In [20]:
image_size = 15
num_labels = 6

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (210, 225) (210, 6)
Validation set (45, 225) (45, 6)
Test set (45, 225) (45, 6)


In [22]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 210
n_layers = 10
graph = tf.Graph()
with graph.as_default():

    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights_1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, n_layers]))
    weights_2 = tf.Variable(
        tf.truncated_normal([n_layers, num_labels]))
    biases_1 = tf.Variable(tf.zeros([n_layers]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    saver = tf.train.Saver({"weights_1":weights_1, "biases_1":biases_1,"weights_2":weights_2, "biases_2":biases_2})

    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits = tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1), weights_2) + biases_2
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2)

In [23]:
num_steps = 100

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases. 
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        
        if (step % 10 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(
            predictions, train_labels[:train_subset, :]))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
            print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
    save_path = saver.save(session, "tmp/model.ckpt")
    print(save_path)

Initialized
Loss at step 0: 19.511900
Training accuracy: 1.4%
Validation accuracy: 37.8%
Test accuracy: 31.1%
Loss at step 10: 0.810045
Training accuracy: 64.8%
Validation accuracy: 75.6%
Test accuracy: 75.6%
Loss at step 20: 0.600134
Training accuracy: 79.0%
Validation accuracy: 71.1%
Test accuracy: 77.8%
Loss at step 30: 0.438964
Training accuracy: 82.4%
Validation accuracy: 82.2%
Test accuracy: 82.2%
Loss at step 40: 0.245151
Training accuracy: 91.9%
Validation accuracy: 86.7%
Test accuracy: 82.2%
Loss at step 50: 0.165371
Training accuracy: 96.2%
Validation accuracy: 88.9%
Test accuracy: 86.7%
Loss at step 60: 0.116348
Training accuracy: 97.1%
Validation accuracy: 91.1%
Test accuracy: 88.9%
Loss at step 70: 0.084523
Training accuracy: 98.1%
Validation accuracy: 93.3%
Test accuracy: 91.1%
Loss at step 80: 0.062695
Training accuracy: 99.0%
Validation accuracy: 93.3%
Test accuracy: 91.1%
Loss at step 90: 0.048445
Training accuracy: 100.0%
Validation accuracy: 93.3%
Test accuracy: 91.1